In [257]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import math

## Dataset Generation - Averaged Version

We don't have a single data set containing all the information we need for the algorithm so we will generate a dataset that fits our needs based on real data.<br>The following datasets appear to have matching observations based on the ItineraryID column, but none contain all the information we need. <br><br>Ticket has a distance, FarePerMile and Itinerary fare per flight, however it does not include the destinations for some reason, while the coupon dataset does.<br>Similarly the time dataset contains the appropriate delay and time data for flights from point A and point B. Note that it had 120 columns with a bunch of useless columns and empty data so I pruned it manually when I had a quick look. <br><br> Using all this we will create a single data set simulating all the data required to model the problem for our algorithm.<br><br>
This particular notebook generates a similar dataset to the full version, but much smaller, since it averages out the price and all times associated with all the individual flights between an origin and a destination airport, effectively creating a single flight between these airports.

In [258]:
ticket = pd.read_csv('Data/ticket_2019_1/ticket_2019_1.csv')
coupon = pd.read_csv('Data/coupon_2019_1/coupon_2019_1.csv')
time = pd.read_csv('Data/time_2019_1/time_2019_1.csv')

Drop columns we don't need. Note all data points are from the US.

In [259]:
ticket = ticket.drop(
    columns=[
        'Coupons',
        'Year',
        'Quarter',
        'OriginAirportID',
        'OriginAirportSeqID',
        'OriginCityMarketID',
        'OriginStateFips',
        'OriginStateName',
        'OriginState',
        'OriginCountry',
        'OriginWac',
        'DollarCred',
        'Passengers',
        'BulkFare',
        'DistanceGroup',
        'ItinGeoType',
        'RPCarrier',
        'OnLine',
        'MilesFlown',
        'FarePerMile',
        'Distance'
    ]
)

coupon = coupon.drop(
    columns=[
        'MktID',
        'SeqNum',
        'Coupons',
        'OriginAirportID',
        'OriginAirportSeqID',
        'OriginCityMarketID',
        'Quarter',
        'OriginCountry',
        'OriginStateFips',
        'OriginWac',
        'DestAirportID',
        'DestAirportSeqID',
        'DestCityMarketID',
        'DestCountry',
        'DestStateFips',
        'DestWac',
        'CouponType',
        'Passengers',
        'FareClass',
        'Gateway',
        'ItinGeoType',
        'RPCarrier',
        'CouponGeoType',
        'TkCarrier',
        'OpCarrier',
        'DistanceGroup',
        'Year',
        'Break',
        'OriginStateName',
        'OriginState',
        'DestStateName',
        'DestState',
        'Distance'
    ]
)


In [260]:
coupon.head()

,ItinID,Origin,Dest
0,201915.0,ABE,ATL
1,201916.0,ABE,ATL
2,201917.0,ABE,ATL
3,201919.0,ABE,ATL
4,2019130.0,ABE,ATL


In [261]:
ticket.head()

,ItinID,Origin,RoundTrip,ItinFare
0,201914.0,ABE,0,203
1,201915.0,ABE,0,211
2,201916.0,ABE,0,241
3,201917.0,ABE,0,266
4,201919.0,ABE,0,314


We'll remove the round trips, since we're only interested in flights from A to B.

In [262]:
print("Percent of Round Trips : {:.2f}%".format((len(ticket.loc[ticket['RoundTrip'] == 1]) / len(ticket['RoundTrip']) * 100), 2))

Percent of Round Trips : 50.01%


In [263]:
ticket = ticket[ticket['RoundTrip']==0]
ticket = ticket.drop(columns=['RoundTrip'])
df = pd.merge(ticket, coupon, on=['ItinID', 'Origin']) 
df

,ItinID,Origin,ItinFare,Dest
0,2.019150e+05,ABE,211,ATL
1,2.019160e+05,ABE,241,ATL
2,2.019170e+05,ABE,266,ATL
3,2.019190e+05,ABE,314,ATL
4,2.019191e+06,ABE,148,ATL
...,...,...,...,...
53997,2.019200e+10,SFO,142,LAX
53998,2.019200e+10,SFO,262,LAX
53999,2.019200e+10,SFO,339,LAX
54000,2.019200e+10,SFO,107,LAX


In [264]:
df = df.drop(columns=['ItinID'])
df.head()

,Origin,ItinFare,Dest
0,ABE,211,ATL
1,ABE,241,ATL
2,ABE,266,ATL
3,ABE,314,ATL
4,ABE,148,ATL


Since we have several values for price for each origin to destination, we could simply consider the average of a flight between these locations.


In [265]:
df = df.groupby(['Origin', 'Dest'])['ItinFare'].mean().reset_index()

In [266]:
df = df.drop_duplicates()
df.describe()

,ItinFare
count,1207.000000
mean,382.351325
std,280.429069
min,4.000000
25%,226.923984
50%,297.983333
75%,455.691297
max,5464.500000


In [267]:
df

,Origin,Dest,ItinFare
0,ABE,ATL,197.392857
1,ABE,PHL,197.000000
2,ABQ,DFW,251.936508
3,ABQ,ORD,194.833333
4,AEX,ATL,324.875000
...,...,...,...
1202,TYS,LGA,308.000000
1203,TYS,MSP,287.333333
1204,XNA,ATL,308.937500
1205,XNA,LGA,527.555556


Now to add the time data into it.

In [268]:
time.head()

,Origin,Dest,DepTime,DepDelay,ArrTime,ArrDelay,ActualElapsedTime
0,SBP,SFO,1353.0,-7.0,1444.0,-26.0,51.0
1,IAH,XNA,930.0,-5.0,1119.0,1.0,109.0
2,SGF,IAH,637.0,-6.0,838.0,-17.0,121.0
3,ISN,DEN,1314.0,-21.0,1404.0,-29.0,110.0
4,MKE,DEN,826.0,-10.0,1009.0,-31.0,163.0


In [269]:
# df = pd.concat([df, time], axis=1, join="inner")
df = pd.merge(df, time, on=['Origin', 'Dest']) 
df.dropna(inplace=True)
df

,Origin,Dest,ItinFare,DepTime,DepDelay,ArrTime,ArrDelay,ActualElapsedTime
0,ABE,ATL,197.392857,557.0,-3.0,819.0,-9.0,142.0
1,ABE,ATL,197.392857,600.0,0.0,825.0,-7.0,145.0
2,ABE,ATL,197.392857,554.0,-6.0,803.0,-28.0,129.0
3,ABE,ATL,197.392857,558.0,-2.0,824.0,-7.0,146.0
4,ABE,ATL,197.392857,555.0,-5.0,813.0,-18.0,138.0
...,...,...,...,...,...,...,...,...
229198,XNA,MSP,452.500000,1834.0,95.0,2036.0,98.0,122.0
229199,XNA,MSP,452.500000,1649.0,-10.0,1852.0,-6.0,123.0
229200,XNA,MSP,452.500000,1651.0,-8.0,1844.0,-14.0,113.0
229201,XNA,MSP,452.500000,1753.0,54.0,1947.0,49.0,114.0


Again, we can average the values to simulate a single flight.

In [270]:
df = df.groupby(['Origin', 'Dest']).mean().reset_index()

In [271]:
df

,Origin,Dest,ItinFare,DepTime,DepDelay,ArrTime,ArrDelay,ActualElapsedTime
0,ABE,ATL,197.392857,1152.029851,7.059701,1350.402985,3.970149,136.880597
1,ABE,PHL,197.000000,968.689655,-2.568966,1057.448276,-3.275862,48.068966
2,ABQ,DFW,251.936508,1147.602804,0.700935,1408.504673,-0.327103,105.200935
3,ABQ,ORD,194.833333,1005.009804,18.147059,1387.774510,14.509804,169.431373
4,AEX,ATL,324.875000,1207.500000,-4.796296,1473.416667,-11.074074,100.361111
...,...,...,...,...,...,...,...,...
1194,TYS,LGA,308.000000,1015.704545,9.750000,1217.068182,2.227273,120.454545
1195,TYS,MSP,287.333333,765.400000,5.933333,924.433333,-5.833333,153.700000
1196,XNA,ATL,308.937500,1224.732919,3.006211,1469.757764,-0.614907,103.782609
1197,XNA,LGA,527.555556,1408.552239,32.104478,1737.373134,34.865672,178.671642


Just to make working with the data easier I'll convert the times to actual time datatype. 

In [272]:
df['DepTime'] = df['DepTime'].astype(int)
df['DepDelay'] = df['DepDelay'].astype(int)
df['ArrTime'] = df['ArrTime'].astype(int)
df['DepDelay'] = df['DepDelay'].astype(int)
df['ArrDelay'] = df['ArrDelay'].astype(int)
df['ActualElapsedTime'] = df['ActualElapsedTime'].astype(int)

def help(time):
    if time >= 2400:
        time -= 100
    if time < 100:
        time = 100
        
    str_time = str(time)
    
    if len(str_time) < 4:
        hours = int(str_time[0]) 
        mins = int(str_time[1:3])
        if(hours > 23): hours = 23
        if(hours < 0): hours = 1
        if(mins > 59): mins = 59
        if(mins < 0): hours = 1
        return dt.time(hour=hours, minute=mins)
    else:
        hours = int(str_time[0:2]) 
        mins = int(str_time[2:4])
        if(hours > 23): hours = 23
        if(hours < 0): hours = 1
        if(mins > 59): mins = 59
        if(mins < 0): hours = 1
        return dt.time(hour=hours, minute=mins)

def converTime(df, col):
    return pd.Series(
        [ help(time) for time in df[col] ]
    )



In [273]:
df['DepTime'] = converTime(df, 'DepTime')
df['ArrTime'] = converTime(df, 'ArrTime')
df.dropna(inplace=True)

Creating a seperate csv that includes longitude and lattitude data for visualization

In [274]:
import chardet
with open('Data/OpenFlights/airports.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
    
airports = pd.read_csv('Data/OpenFlights/airports.csv', encoding=result['encoding'])

In [275]:
# aiports = airports[airports['Country']=='United States']
airports.drop(airports.loc[airports['Country']!='United States'].index, inplace=True)
airports.reset_index()
aiports = airports.drop(columns=['Name', 'City', 'ICAO', 'Country'])

In [279]:
locations = df.merge(airports,on=['Origin'])
locations = locations.dropna()
locations = locations.drop(columns=['Dest', 'ItinFare', 'DepDelay', 'ArrDelay', 'ActualElapsedTime', 'Name', 'City', 'Country', 'ICAO', 'DepTime', 'ArrTime'])
locations

,Origin,Lat,Long
0,ABE,40.652100,-75.440804
1,ABE,40.652100,-75.440804
2,ABQ,35.040199,-106.609001
3,ABQ,35.040199,-106.609001
4,AEX,31.327400,-92.549797
...,...,...,...
1184,TYS,35.811001,-83.994003
1185,TYS,35.811001,-83.994003
1186,XNA,36.281898,-94.306801
1187,XNA,36.281898,-94.306801


In [280]:
locations.to_csv(os.getcwd() + '\\Data\\processed\\airportLocations.csv', index=False)

In [281]:
df = df[(df['Origin'].isin(locations['Origin']))&(df['Dest'].isin(locations['Origin']))]
df.to_csv(os.getcwd() + '\\Data\\processed\\flightData.csv', index=False)
df

,Origin,Dest,ItinFare,DepTime,DepDelay,ArrTime,ArrDelay,ActualElapsedTime
0,ABE,ATL,197.392857,11:52:00,7,13:50:00,3,136
1,ABE,PHL,197.000000,09:59:00,-2,10:57:00,-3,48
2,ABQ,DFW,251.936508,11:47:00,0,14:08:00,0,105
3,ABQ,ORD,194.833333,10:05:00,18,13:59:00,14,169
4,AEX,ATL,324.875000,12:07:00,-4,14:59:00,-11,100
...,...,...,...,...,...,...,...,...
1194,TYS,LGA,308.000000,10:15:00,9,12:17:00,2,120
1195,TYS,MSP,287.333333,07:59:00,5,09:24:00,-5,153
1196,XNA,ATL,308.937500,12:24:00,3,14:59:00,0,103
1197,XNA,LGA,527.555556,14:08:00,32,17:37:00,34,178
